# 데이터에 대한 이해와 전처리

In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.utils import to_categorical

In [2]:
text="""
한현영은 한국에서 태어나 디지스트에 재학중인 대학생이다\n
우리 팀은 유지알피 1등을 할 것이다\n
한현영은 인천 출신이다\n
인공지능 대중화를 위한 프레임워크 샌디\n
인공지능은 정말 멋있다\n"""

단어 집합을 생성하고 크기를 확인한다.

In [3]:
t = Tokenizer()
t.fit_on_texts([text])
vocab_size = len(t.word_index) + 1
# 케라스 토크나이저의 정수 인코딩은 인덱스가 1부터 시작하지만,
# 케라스 원-핫 인코딩에서 배열의 인덱스가 0부터 시작하기 때문에
# 배열의 크기를 실제 단어 집합의 크기보다 +1로 생성해야하므로 미리 +1 선언 
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 23


각 단어와 단어에 부여된 정수 index를 출력한다.

In [4]:
print(t.word_index)

{'한현영은': 1, '한국에서': 2, '태어나': 3, '디지스트에': 4, '재학중인': 5, '대학생이다': 6, '우리': 7, '팀은': 8, '유지알피': 9, '1등을': 10, '할': 11, '것이다': 12, '인천': 13, '출신이다': 14, '인공지능': 15, '대중화를': 16, '위한': 17, '프레임워크': 18, '샌디': 19, '인공지능은': 20, '정말': 21, '멋있다': 22}


훈련 데이터를 만들어본다.

In [5]:
sequences = list()
for line in text.split('\n'): # Wn을 기준으로 문장 토큰화
    encoded = t.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

print('학습에 사용할 샘플의 개수: %d' % len(sequences))

학습에 사용할 샘플의 개수: 18


샘플의 개수는 총 11개이다. 전체 샘플을 출력해보자.

In [6]:
print(sequences)

[[1, 2], [1, 2, 3], [1, 2, 3, 4], [1, 2, 3, 4, 5], [1, 2, 3, 4, 5, 6], [7, 8], [7, 8, 9], [7, 8, 9, 10], [7, 8, 9, 10, 11], [7, 8, 9, 10, 11, 12], [1, 13], [1, 13, 14], [15, 16], [15, 16, 17], [15, 16, 17, 18], [15, 16, 17, 18, 19], [20, 21], [20, 21, 22]]


위의 데이터는 아직 레이블로 사용될 단어를 분리하지 않은 훈련 데이터이다. [2, 3]은 [경마장에, 있는]에 해당되며 [2, 3, 1]은 [경마장에, 있는, 말이]에 해당된다. 전체 훈련 데이터에 대해서 맨 우측에 있는 단어에 대해서만 레이블로 분리해야 한다.

우선 전체 샘플에 대해서 길이를 일치시켜 준다. 가장 긴 샘플의 길이를 기준으로 한다. 현재 육안으로 봤을 때, 길이가 가장 긴 샘플은 [8, 1, 9, 10, 1, 11]이고 길이는 6이다. 이를 코드로는 다음과 같이 구할 수 있다.

In [7]:
max_len=max(len(l) for l in sequences) # 모든 샘플에서 길이가 가장 긴 샘플의 길이 출력
print('샘플의 최대 길이 : {}'.format(max_len))

샘플의 최대 길이 : 6


전체 훈련 데이터에서 가장 긴 샘플의 길이가 6임을 확인했다. 따라서 전체 샘플의 길이를 6으로 padding한다.

In [8]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')

pad_sequences()는 모든 샘플에 대해서 0을 사용하여 길이를 맞춰줍니다. maxlen의 값으로 6을 주면 모든 샘플의 길이를 6으로 맞춰주며, padding의 인자로 'pre'를 주면 길이가 6보다 짧은 샘플의 앞에 0으로 채웁니다. 전체 훈련 데이터를 출력해봅니다.

In [9]:
print(sequences)

[[ 0  0  0  0  1  2]
 [ 0  0  0  1  2  3]
 [ 0  0  1  2  3  4]
 [ 0  1  2  3  4  5]
 [ 1  2  3  4  5  6]
 [ 0  0  0  0  7  8]
 [ 0  0  0  7  8  9]
 [ 0  0  7  8  9 10]
 [ 0  7  8  9 10 11]
 [ 7  8  9 10 11 12]
 [ 0  0  0  0  1 13]
 [ 0  0  0  1 13 14]
 [ 0  0  0  0 15 16]
 [ 0  0  0 15 16 17]
 [ 0  0 15 16 17 18]
 [ 0 15 16 17 18 19]
 [ 0  0  0  0 20 21]
 [ 0  0  0 20 21 22]]


In [10]:
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]
# 리스트의 마지막 값을 제외하고 저장한 것은 X
# 리스트의 마지막 값만 저장한 것은 y. 이는 레이블에 해당됨.

In [11]:
print(X)

[[ 0  0  0  0  1]
 [ 0  0  0  1  2]
 [ 0  0  1  2  3]
 [ 0  1  2  3  4]
 [ 1  2  3  4  5]
 [ 0  0  0  0  7]
 [ 0  0  0  7  8]
 [ 0  0  7  8  9]
 [ 0  7  8  9 10]
 [ 7  8  9 10 11]
 [ 0  0  0  0  1]
 [ 0  0  0  1 13]
 [ 0  0  0  0 15]
 [ 0  0  0 15 16]
 [ 0  0 15 16 17]
 [ 0 15 16 17 18]
 [ 0  0  0  0 20]
 [ 0  0  0 20 21]]


In [12]:
print(y)

[ 2  3  4  5  6  8  9 10 11 12 13 14 16 17 18 19 21 22]


레이블이 분리되었다. RNN 모델에 훈련 데이터를 훈련시키기 전에 레이블에 대해 One-Hot encoding을 수행한다.

In [13]:
y = to_categorical(y, num_classes=vocab_size)

In [14]:
print(y)

[[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.

# 모델 설계

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

In [16]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_len-1)) # 레이블을 분리하였으므로 이제 X의 길이는 5
model.add(SimpleRNN(32))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)

Epoch 1/200
1/1 - 0s - loss: 3.1323 - accuracy: 0.0000e+00
Epoch 2/200
1/1 - 0s - loss: 3.1244 - accuracy: 0.0556
Epoch 3/200
1/1 - 0s - loss: 3.1165 - accuracy: 0.1111
Epoch 4/200
1/1 - 0s - loss: 3.1088 - accuracy: 0.1111
Epoch 5/200
1/1 - 0s - loss: 3.1011 - accuracy: 0.1111
Epoch 6/200
1/1 - 0s - loss: 3.0933 - accuracy: 0.1111
Epoch 7/200
1/1 - 0s - loss: 3.0856 - accuracy: 0.1111
Epoch 8/200
1/1 - 0s - loss: 3.0777 - accuracy: 0.1111
Epoch 9/200
1/1 - 0s - loss: 3.0697 - accuracy: 0.1111
Epoch 10/200
1/1 - 0s - loss: 3.0615 - accuracy: 0.1111
Epoch 11/200
1/1 - 0s - loss: 3.0531 - accuracy: 0.1111
Epoch 12/200
1/1 - 0s - loss: 3.0444 - accuracy: 0.1111
Epoch 13/200
1/1 - 0s - loss: 3.0355 - accuracy: 0.1111
Epoch 14/200
1/1 - 0s - loss: 3.0263 - accuracy: 0.1111
Epoch 15/200
1/1 - 0s - loss: 3.0167 - accuracy: 0.1111
Epoch 16/200
1/1 - 0s - loss: 3.0068 - accuracy: 0.1667
Epoch 17/200
1/1 - 0s - loss: 2.9965 - accuracy: 0.1667
Epoch 18/200
1/1 - 0s - loss: 2.9857 - accuracy: 0.16

Epoch 147/200
1/1 - 0s - loss: 0.6797 - accuracy: 0.9444
Epoch 148/200
1/1 - 0s - loss: 0.6711 - accuracy: 0.9444
Epoch 149/200
1/1 - 0s - loss: 0.6626 - accuracy: 0.9444
Epoch 150/200
1/1 - 0s - loss: 0.6542 - accuracy: 0.9444
Epoch 151/200
1/1 - 0s - loss: 0.6460 - accuracy: 0.9444
Epoch 152/200
1/1 - 0s - loss: 0.6378 - accuracy: 0.9444
Epoch 153/200
1/1 - 0s - loss: 0.6298 - accuracy: 0.9444
Epoch 154/200
1/1 - 0s - loss: 0.6218 - accuracy: 0.9444
Epoch 155/200
1/1 - 0s - loss: 0.6140 - accuracy: 0.9444
Epoch 156/200
1/1 - 0s - loss: 0.6063 - accuracy: 0.9444
Epoch 157/200
1/1 - 0s - loss: 0.5987 - accuracy: 0.9444
Epoch 158/200
1/1 - 0s - loss: 0.5912 - accuracy: 0.9444
Epoch 159/200
1/1 - 0s - loss: 0.5839 - accuracy: 0.9444
Epoch 160/200
1/1 - 0s - loss: 0.5766 - accuracy: 0.9444
Epoch 161/200
1/1 - 0s - loss: 0.5694 - accuracy: 0.9444
Epoch 162/200
1/1 - 0s - loss: 0.5624 - accuracy: 0.9444
Epoch 163/200
1/1 - 0s - loss: 0.5554 - accuracy: 0.9444
Epoch 164/200
1/1 - 0s - loss: 

각 단어의 embedding vector는 10차원을 가지고, 32의 은닉 상태 크기를 가지는 RNN을 사용한다.

In [17]:
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=5, padding='pre') # 데이터에 대한 패딩
        result = model.predict_classes(encoded, verbose=0)
    # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' '  + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
    # for문이므로 이 행동을 다시 반복
    sentence = init_word + sentence
    return sentence

In [18]:
print(sentence_generation(model, t, '우리', 6))

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
우리 팀은 유지알피 1등을 할 것이다 것이다


In [19]:
for i in range(10):
    print(sentence_generation(model, t, '한현영은', i))

한현영은
한현영은 인천
한현영은 인천 출신이다
한현영은 인천 출신이다 디지스트에
한현영은 인천 출신이다 디지스트에 재학중인
한현영은 인천 출신이다 디지스트에 재학중인 대학생이다
한현영은 인천 출신이다 디지스트에 재학중인 대학생이다 대학생이다
한현영은 인천 출신이다 디지스트에 재학중인 대학생이다 대학생이다 것이다
한현영은 인천 출신이다 디지스트에 재학중인 대학생이다 대학생이다 것이다 것이다
한현영은 인천 출신이다 디지스트에 재학중인 대학생이다 대학생이다 것이다 것이다 대학생이다


In [20]:
print(sentence_generation(model, t, '인공지능은', 4))

인공지능은 정말 멋있다 디지스트에 재학중인
